In [315]:
import tweepy
import pandas as pd
import yfinance as yf
import openai
import big_baller_moves
import json
import requests

In [316]:
#Put your Bearer Token in the parenthesis below
client1 = tweepy.Client(bearer_token=big_baller_moves.bossman_tingz["twitter_api_key"])

# Authenticate
auth = tweepy.OAuth1UserHandler(big_baller_moves.bossman_tingz["consumer_key"], big_baller_moves.bossman_tingz["consumer_secret"], big_baller_moves.bossman_tingz["access_token"], 
                                big_baller_moves.bossman_tingz["access_token_secret"])

# Create API object
api = tweepy.API(auth)

# openai key
openai.api_key = big_baller_moves.bossman_tingz["openai_api_key"]

In [317]:
#enter username to get the user ID
username = "Heisenberg_100k"

try:
    user = client1.get_user(username=username)
    user_id = user_id = user.data.id
    print("User ID for", username, ":", user_id)
except Exception as e:
    print("Error:", e)

User ID for Heisenberg_100k : 1750349614786691072


In [319]:
#get btc price (not working atm)

# Choose Token and Time
timestamp_open = pd.Timestamp('2024-05-02 05:09:00', tz='UTC')
unix_timestamp = int(timestamp_open.timestamp())
unix_timestamp_24 = unix_timestamp + (24*60*60)
unix_timestamp_48 = unix_timestamp + (48*60*60)
ckey = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp}&localization=false"
url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"
ckey24 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_24}&localization=false"
ckey48 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_48}&localization=false"

try:
        # Convert timestamp to Unix timestamp (Kraken API requires timestamp in seconds)
        unix_timestamp = int(timestamp.timestamp())

        # Kraken API endpoint for OHLC data
        url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"

response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'result' in data and 'XBTUSD' in data['result'] and data['result']['XBTUSD']:
                # The response contains a list of lists, where each inner list represents OHLC data for a specific interval
                # We extract the closing price from the last list (most recent data point)
                bitcoin_price = float(data['result']['XBTUSD'][-1][4])
                return bitcoin_price
            else:
                print("No data available for the specified timestamp")
                return None
        else:
            print(f"Failed to retrieve data from Kraken API. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


IndentationError: unindent does not match any outer indentation level (<string>, line 34)

In [252]:
unix_timestamp

1714626540

In [320]:
#get all tweets
tweets = client1.get_users_tweets(
    id= user_id,
    max_results=100,  # Number of tweets to retrieve (adjust as needed)
    tweet_fields=['id', 'text', 'created_at', 'entities', 'attachments'],  # Fields you want to retrieve for each tweet
    media_fields=['preview_image_url', 'url'],
    exclude = ['retweets','replies'],
    expansions=['attachments.media_keys', 'author_id']
)


In [321]:
# Process retrieved tweets

for tweet in tweets.data:
    print(f"Tweet ID: {tweet.id}, Text: {tweet.text}, Created At: {tweet.created_at}")


Tweet ID: 1790077422697693598, Text: Question:

Sell 1 naked GME call here.

Worth it?

Dangerous?

Only 1 shouldn’t be too bad?, Created At: 2024-05-13 17:51:30+00:00
Tweet ID: 1790077076105498901, Text: $PARA Open

See how Sony plays the buyout process.

Small. https://t.co/CceV23s8gb, Created At: 2024-05-13 17:50:07+00:00
Tweet ID: 1790026595027005823, Text: Sold my $INTC calls too soon. But shares doing well.

Anyone hopped in the INTC trade?, Created At: 2024-05-13 14:29:31+00:00
Tweet ID: 1790014939173630180, Text: $VKTX Open https://t.co/2G1TFQ8iaV, Created At: 2024-05-13 13:43:13+00:00
Tweet ID: 1790014760244621631, Text: $INTC Closed, switched to shares

In 1.15 (May 8th)

Out 1.29

+12.1% 
+$140

Switched to 400 shares instead. Still looking for 32-33’s. https://t.co/LeXt1fPshM, Created At: 2024-05-13 13:42:30+00:00
Tweet ID: 1789972227565273090, Text: $VKTX and $SMMT back on watch., Created At: 2024-05-13 10:53:29+00:00
Tweet ID: 1789968454356779401, Text: $GME selling call 

In [339]:
tweets

Response(data=[<Tweet id=1790077422697693598 text='Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad?'>, <Tweet id=1790077076105498901 text='$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb'>, <Tweet id=1790026595027005823 text='Sold my $INTC calls too soon. But shares doing well.\n\nAnyone hopped in the INTC trade?'>, <Tweet id=1790014939173630180 text='$VKTX Open https://t.co/2G1TFQ8iaV'>, <Tweet id=1790014760244621631 text='$INTC Closed, switched to shares\n\nIn 1.15 (May 8th)\n\nOut 1.29\n\n+12.1% \n+$140\n\nSwitched to 400 shares instead. Still looking for 32-33’s. https://t.co/LeXt1fPshM'>, <Tweet id=1789972227565273090 text='$VKTX and $SMMT back on watch.'>, <Tweet id=1789968454356779401 text='$GME selling call spreads on watch.'>, <Tweet id=1789302719662158311 text='You know it is true. https://t.co/HRF5z5GmYK'>, <Tweet id=1789299906559262797 text='The volatility of volatility $VVIX is essent

In [322]:
def get_display_url(entities):
    if isinstance(entities, dict) and "urls" in entities:
        urls = entities["urls"]
        display_urls = [url.get("display_url") for url in urls]
        return display_urls
    else:
        return 0

# Apply the function to count display_url
df["image_url"] = df["entities"].apply(get_display_url)

selected_columns = ['id', 'text', 'created_at', 'image_url']

# Creating a new DataFrame with selected columns
heisenberg_tweets = df[selected_columns].copy()

In [314]:
heisenberg_tweets

,id,text,created_at,image_url,Image
0,1790077422697693598,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad?,2024-05-13 17:51:30+00:00,None,No image available
1,1790077076105498901,$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb,2024-05-13 17:50:07+00:00,None,None
2,1790026595027005823,Sold my $INTC calls too soon. But shares doing well.\n\nAnyone hopped in the INTC trade?,2024-05-13 14:29:31+00:00,None,No image available
3,1790014939173630180,$VKTX Open https://t.co/2G1TFQ8iaV,2024-05-13 13:43:13+00:00,None,None
4,1790014760244621631,"$INTC Closed, switched to shares\n\nIn 1.15 (May 8th)\n\nOut 1.29\n\n+12.1% \n+$140\n\nSwitched to 400 shares instead. Still looking for 32-33’s. https://t.co/LeXt1fPshM",2024-05-13 13:42:30+00:00,None,None
...,...,...,...,...,...
93,1783854096040292837,$SOXL Closed\n\nIn 32.38 (April 19)\n\nOut 37.57\n\nGood enough https://t.co/fmM5DMrJan,2024-04-26 13:42:13+00:00,None,None
94,1783543215695753574,$META 440. \n\nDo it for daddy.,2024-04-25 17:06:53+00:00,None,No image available
95,1783514573951365454,$META Need a GOOG and a MSFT beat to get META going tomorrow.\n\n🤞,2024-04-25 15:13:05+00:00,None,No image available
96,1783230723195052350,$META failed.\n\nWe gloat the winners.\n\nWe accept defeats on the losers.\n\nNeed a miracle by Friday. Otherwise -99%.,2024-04-24 20:25:09+00:00,None,No image available


In [256]:
df = pd.DataFrame(tweets.data)

In [273]:
#helper counting function to check display_urls

def count_display_url(entities):
    if isinstance(entities, dict) and "urls" in entities:
        urls = entities["urls"]
        display_urls = [url.get("display_url") for url in urls]
        print(display_urls)
        return sum(1 for url in display_urls if url is not None)
    else:
        return 0

# Apply the function to count display_url
count = df["entities"].apply(count_display_url).sum()

print("Number of rows with display_url:", count)

['pic.twitter.com/CceV23s8gb']
['pic.twitter.com/2G1TFQ8iaV']
['pic.twitter.com/LeXt1fPshM', 'pic.twitter.com/LeXt1fPshM']
['pic.twitter.com/HRF5z5GmYK']
['x.com/i/web/status/1…']
['x.com/i/web/status/1…']
['pic.twitter.com/FJwJ8S1RdE']
['x.com/i/web/status/1…', 'x.com/Heisenberg_100…']
['pic.twitter.com/shmUd4IV7w']
['pic.twitter.com/1FrUXsjpbc']
['pic.twitter.com/FsZa8r2C56']
['pic.twitter.com/WNy2WINVwq']
['pic.twitter.com/wZl8ISwGxb']
['pic.twitter.com/iigSxnAU8S']
['pic.twitter.com/qsWN4xdihq']
['pic.twitter.com/rL3Y94Ex9E']
['x.com/i/web/status/1…']
['x.com/i/web/status/1…', 'x.com/ShiKitYip1/sta…']
['pic.twitter.com/aIjR48ncY0']
['pic.twitter.com/64syimBVcM']
['pic.twitter.com/bMkPWxmKx4']
['x.com/i/web/status/1…']
['pic.twitter.com/9Wo5JfC8mO']
['pic.twitter.com/pm160UzPGD', 'pic.twitter.com/pm160UzPGD', 'pic.twitter.com/pm160UzPGD']
['pic.twitter.com/xtQMTvDNEY']
['pic.twitter.com/CcLC9F7DSF']
['pic.twitter.com/UproWGtGsH']
['pic.twitter.com/RFpL6lKTGj']
['pic.twitter.com/Wvnh

In [271]:
df[df['attachments'].isnull()]['entities']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [41]:
ticker_symbol = 'AAPL'

# Fetch historical data
stock_data = yf.Ticker(ticker_symbol)

# Get historical market data
historical_data = stock_data.history(period="max")

print(historical_data)

                                 Open        High         Low       Close  \
Date                                                                        
1980-12-12 00:00:00-05:00    0.099192    0.099623    0.099192    0.099192   
1980-12-15 00:00:00-05:00    0.094448    0.094448    0.094017    0.094017   
1980-12-16 00:00:00-05:00    0.087548    0.087548    0.087117    0.087117   
1980-12-17 00:00:00-05:00    0.089273    0.089704    0.089273    0.089273   
1980-12-18 00:00:00-05:00    0.091861    0.092292    0.091861    0.091861   
...                               ...         ...         ...         ...   
2024-04-29 00:00:00-04:00  173.369995  176.029999  173.100006  173.500000   
2024-04-30 00:00:00-04:00  173.330002  174.990005  170.000000  170.330002   
2024-05-01 00:00:00-04:00  169.580002  172.710007  169.110001  169.300003   
2024-05-02 00:00:00-04:00  172.509995  173.419998  170.889999  173.029999   
2024-05-06 00:00:00-04:00  182.354004  184.169998  181.630005  181.835007   

In [75]:
client = openai
df1 = df.copy().head(5)
pd.set_option('display.max_colwidth', None)

In [76]:
#for astronomer
system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [ ]:
#heisenberg
system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [96]:
def get_response(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Tell me if the message is suggesting opening a long, short or N/A in one word. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short." }, 
                                                        {"role":"user","content": text}])
   
    return response.choices[0].message.content.strip()



In [106]:
def get_response_guided(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Your job is to tag tweets as either opening a long position or short position or N/A. Provide a reason explaining why the tweet is a long or short or n/a." }, 
                                                        {"role":"user","content": "Taking profit"},
                                                        {"role":"assistant","content": "Short, the poster is taking profit."}])
   
    return response.choices[0].message.content.strip()


In [326]:
from bs4 import BeautifulSoup
from selenium import webdriver



def get_response_image(text) :
    if text == 0:
        return "No image available"
    if not isinstance(text, str):
        print(text)
        text = text[0]
    try:
        response = client.chat.completions.create(model="gpt-4o", 
                                                   messages=[
        {
          "role": "user",
          "content": [
            {"type": "text", "text": "What option play is this image describing?"},
            {
              "type": "image_url",
              "image_url": {
                "url": text,
              },
            },
          ],
        }
      ],
      max_tokens=300,)
   
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass

In [107]:
df1['Direction (Guided)'] = df1['text'].apply(get_response_guided)
df1['Direction'] = df1['text'].apply(get_response)

In [327]:

heisenberg_tweets['Image'] = heisenberg_tweets['image_url'].apply(get_response_image)


['pic.twitter.com/CceV23s8gb']


InvalidArgumentException: Message: invalid argument
  (Session info: chrome=124.0.6367.60)
Stacktrace:
	GetHandleVerifier [0x00007FF6553D1522+60802]
	(No symbol) [0x00007FF65534AC22]
	(No symbol) [0x00007FF655207B13]
	(No symbol) [0x00007FF6551F05E0]
	(No symbol) [0x00007FF6551EEC3D]
	(No symbol) [0x00007FF6551EF22C]
	(No symbol) [0x00007FF65520A9F9]
	(No symbol) [0x00007FF65529AB7E]
	(No symbol) [0x00007FF65527AB7A]
	(No symbol) [0x00007FF65529A224]
	(No symbol) [0x00007FF65527A923]
	(No symbol) [0x00007FF655248FEC]
	(No symbol) [0x00007FF655249C21]
	GetHandleVerifier [0x00007FF6556D41BD+3217949]
	GetHandleVerifier [0x00007FF655716157+3488183]
	GetHandleVerifier [0x00007FF65570F0DF+3459391]
	GetHandleVerifier [0x00007FF65548B8E6+823622]
	(No symbol) [0x00007FF655355FBF]
	(No symbol) [0x00007FF655350EE4]
	(No symbol) [0x00007FF655351072]
	(No symbol) [0x00007FF6553418C4]
	BaseThreadInitThunk [0x00007FF859D7257D+29]
	RtlUserThreadStart [0x00007FF85A9AAA48+40]


In [108]:

selected = ['text','Direction','Direction (Guided)']
df1[selected]

,text,Direction,Direction (Guided)
0,"#Sentiment check\n\nWhich camp are you ? ⛺️\n\nA: Camp bulls, or \nB: Camp bears ?\n\nGive me your thoughts📊 https://t.co/8hyVQnk3tG",N/A,N/A. The tweet does not provide enough information to determine if the user is opening a long or short position.
1,"$BTC Update\n\nReceived the final squeeze we wanted🎯 prepare for downside\n\nIn last update, called how it's a good time to short around these prices but that weekend highs are likely to come first and how that's a good time to take profit of our big longs we have been building all… https://t.co/zGhswZAQ3r https://t.co/XFxkNagKE3 https://t.co/ma9kIVi8pR",Short,"N/A, as the tweet does not provide enough information to determine whether it is advocating for taking a long or short position."
2,"$BTC...\n\nCould smell that scam pump from a mile away \n\nSo figured was a good point to take advantage of the high prices.\n\nPin point tp. 🎯 Market dropping not one hour later.\n\nLike I said, still short, and back to 62k-60k zone.\n\n#BTC https://t.co/BrfvF8096U https://t.co/n6clGy0uhS",Short,"N/A, the tweet does not provide enough information to determine whether to open a long or short position."
3,"$BTC\n\nHitting it big, at our weekend highs\n\nSecond tp on our longs hit🎯\n\nTaking off another 25 % at 2.1 R 💰\n\nWe were building this position all week last week and doubled down as it moved to 57k because of our expectancy this range being bullish overall.\n\nPretty big position so… https://t.co/e4GnUA0JOJ https://t.co/Qw0K1GgrJH https://t.co/HetGlPXfai",long,"N/A, the tweet does not provide enough information to determine a clear long or short position."
4,"$BTC Update\n\nBullish weekly close, but don't be fooled, I'm still short\n\nLast update, stated how this range is bullish and how we're ultimately going to the upside, inevitably imo. Also talked about the possibility of a small pullback so took a short to decrease long risk and… https://t.co/qcz38nMoGG https://t.co/Ml20sLb2ZO https://t.co/iEmerXOtFG",Short,"N/A, the tweet does not provide enough information to determine if the poster is opening a long or short position."
